In [147]:
import os
from pathlib import Path

from arcgis import GIS
from arcgis.geometry import Geometry
from arcgis.features import FeatureSet
from dotenv import load_dotenv
from modeling import Country
import pandas as pd

In [16]:
dotenv_pth = Path('.').absolute().parent.parent/'.env'
assert dotenv_pth.exists()
_ = load_dotenv(dotenv_pth)

In [22]:
gis = GIS(os.getenv('ESRI_PORTAL_URL'), username=os.getenv('ESRI_PORTAL_USERNAME'), password=os.getenv('ESRI_PORTAL_PASSWORD'))

assert 'geoai-ent' in gis.url

In [28]:
helper_dict = gis.properties.helperServices
[k for k in helper_dict if 'route' in k.lower()]

['asyncRoute', 'route']

In [30]:
gis.properties.helperServices.route.url

'https://geoai-ent.bd.esri.com/baserver/rest/services/Routing/NetworkAnalysis/NAServer/Route'

In [31]:
from modeling import utils

In [33]:
utils.has_networkanalysis_gis(gis.users.me, 'closestfacility')

False

In [37]:
res = gis._con.get(gis.properties.helperServices.route.url)

In [70]:
class Proximity:
    
    def __init__(self, mdl):
        
        self.source = source
        self._travel_modes = None
        self._url = gis.properties.helperServices.route.url
        self._properties = self.source._con.get(gis.properties.helperServices.route.url)
        
    def __repr__(self):
        return f'<Proximity - {gis.__repr__()}>'
        
    @property
    def travel_modes(self)->pd.DataFrame:
        if self._travel_modes is None:
            trvl_df = pd.DataFrame(self._properties['supportedTravelModes'])
            trvl_df['key'] = trvl_df['name'].str.lower().str.replace(' ', '_')
            trvl_df.set_index('key', inplace=True, drop=True)
            self._travel_modes = trvl_df
        return self._travel_modes
    
    def get_nearest(self, destinations: pd.DataFrame, ):
        url = f'{self._url}/solveClosestFacility'
        params = {
            'incidents': origins,
            'facilities': destination, 
            'returnCFRoutes': ,
            'f': 'json',
        }

In [76]:
from arcgis.features import FeatureCollection

In [78]:
cntry = Country('USA')

cntry

<modeling.Country - USA (GIS at https://geoai-ent.bd.esri.com/portal/ logged in as jmccune)>

In [219]:
source = gis

In [223]:
route_url = source.properties.helperServices.route.url
url = '/'.join(route_url.split('/')[:-1]) + '/ClosestFacility/solveClosestFacility'

In [80]:
aoi_df = cntry.cities_and_towns_places.get('seattle')

aoi_df

,ID,NAME,SHAPE
0,5363000,Seattle city,"{""rings"": [[[-122.4052899999732, 47.6665600007..."


In [81]:
origin_df = aoi_df.mdl.census_tracts.get()

origin_df.head()

,ID,NAME,SHAPE
0,53033000100,530330001.00,"{""rings"": [[[-122.29109799990732, 47.733739999..."
1,53033000200,530330002.00,"{""rings"": [[[-122.2964269993105, 47.7279059990..."
2,53033000401,530330004.01,"{""rings"": [[[-122.34501199966812, 47.726327999..."
3,53033000402,530330004.02,"{""rings"": [[[-122.35554800007239, 47.723231999..."
4,53033000500,530330005.00,"{""rings"": [[[-122.36086799925472, 47.712360999..."


In [165]:
dest_df = aoi_df.mdl.business.get_by_name('ace hardware')

dest_df

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,174746248,MAGNOLIA ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SEATTLE,98199,WA,"{""x"": -122.39802, ""y"": 47.6402850000001, ""spat...",174746248,MAGNOLIA ACE HARDWARE,MAGNOLIA ACE HARDWARE
1,426576447,HERO ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SEATTLE,98101,WA,"{""x"": -122.3393535, ""y"": 47.6123850000001, ""sp...",426576447,HERO ACE HARDWARE,HERO ACE HARDWARE
2,664387990,TWEEDY-POPP LAKE CITY ACE HDWR,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SEATTLE,98125,WA,"{""x"": -122.294826, ""y"": 47.7217440000001, ""spa...",664387990,TWEEDY-POPP LAKE CITY ACE HDWR,TWEEDY-POPP LAKE CITY ACE HDWR


In [166]:
origin_df = aoi_df.mdl.census_tracts.get()
origin_df[origin_df.spatial.name] = origin_df[origin_df.spatial.name].apply(lambda geom: Geometry({'x': geom.centroid[0], 'y': geom.centroid[1], 'spatialReference': geom.spatial_reference}))
origin_df.spatial.set_geometry(origin_df.spatial.name)
origin_df.head()

,ID,NAME,SHAPE
0,53033000100,530330001.00,"{""x"": -122.28869144768473, ""y"": 47.72628013426..."
1,53033000200,530330002.00,"{""x"": -122.30999685849585, ""y"": 47.72664028948..."
2,53033000401,530330004.01,"{""x"": -122.35267465290916, ""y"": 47.72894236603..."
3,53033000402,530330004.02,"{""x"": -122.35236661948676, ""y"": 47.71685892324..."
4,53033000500,530330005.00,"{""x"": -122.36811869780774, ""y"": 47.72176669141..."


In [167]:
self = Router(gis)

In [174]:
from modeling.proximity import _prep_sdf_for_nearest, _reformat_closest_result_dataframe, _explode_closest_rank_dataframe

return_geometry = True
output_spatial_reference = 4326

In [212]:
origin_fs = _prep_sdf_for_nearest(origin_df, 'ID').spatial.to_featureset().to_dict()
dest_fs = _prep_sdf_for_nearest(dest_df, 'LOCNUM').spatial.to_featureset().to_dict()

url = '/'.join(self._url.split('/')[:-1]) + '/ClosestFacility/solveClosestFacility'

return_geom = 'esriNAOutputLineTrueShape' if return_geometry else 'esriNAOutputLineNone'

out_sr = {'wkid': output_spatial_reference}

In [224]:
params = {
    'incidents': origin_fs,
    'facilities': dest_fs, 
    'returnCFRoutes': True,
    'f': 'json',
    'outputLines': return_geom,
    'outSR': out_sr
}

In [225]:
res = self.source._con.post(url, params)

In [226]:
route_df = FeatureSet.from_dict(res['routes']).sdf

route_df

,ObjectID,FacilityID,FacilityRank,Name,IncidentCurbApproach,FacilityCurbApproach,IncidentID,StartTime,EndTime,StartTimeUTC,...,Total_Minutes,Total_TravelTime,Total_Miles,Total_Kilometers,Total_TimeAt1KPH,Total_WalkTime,Total_TruckMinutes,Total_TruckTravelTime,Shape_Length,SHAPE
0,1,3,1,53033000100 - 664387990,1,2,1,None,None,None,...,None,2.343517,0.558464,0.898760,None,None,None,None,0.009662,"{""paths"": [[[-122.28870472599999, 47.726279072..."
1,2,3,1,53033000200 - 664387990,1,2,2,None,None,None,...,None,5.453073,1.828058,2.941975,None,None,None,None,0.033657,"{""paths"": [[[-122.30999685799998, 47.726540000..."
2,3,3,1,53033000401 - 664387990,2,2,3,None,None,None,...,None,10.202465,3.492966,5.621384,None,None,None,None,0.069475,"{""paths"": [[[-122.35270325699997, 47.728656326..."
3,4,3,1,53033000402 - 664387990,1,2,4,None,None,None,...,None,9.730853,3.623812,5.831960,None,None,None,None,0.071506,"{""paths"": [[[-122.35294040099996, 47.716877839..."
4,5,3,1,53033000500 - 664387990,2,2,5,None,None,None,...,None,12.103868,4.015186,6.461815,None,None,None,None,0.082098,"{""paths"": [[[-122.36811477399999, 47.721503809..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,129,2,1,53033010900 - 426576447,1,2,129,None,None,None,...,None,15.251560,5.780928,9.303501,None,None,None,None,0.092531,"{""paths"": [[[-122.31947639299995, 47.542259001..."
129,130,2,1,53033011300 - 426576447,2,2,130,None,None,None,...,None,19.134858,7.650741,12.312674,None,None,None,None,0.120394,"{""paths"": [[[-122.34418123699999, 47.527442224..."
130,131,2,1,53033011700 - 426576447,2,2,131,None,None,None,...,None,21.141576,8.098313,13.032971,None,None,None,None,0.133237,"{""paths"": [[[-122.28209657699995, 47.519867344..."
131,132,2,1,53033012000 - 426576447,1,2,132,None,None,None,...,None,22.261617,9.152045,14.728789,None,None,None,None,0.149897,"{""paths"": [[[-122.38067114199998, 47.513736955..."


In [216]:
notna_srs = route_df.isna().all()
drop_cols = notna_srs[notna_srs].index.values
route_df.drop(columns=drop_cols, inplace=True)

In [217]:
std_clstst_df = _reformat_closest_result_dataframe(route_df)

In [218]:
_explode_closest_rank_dataframe(std_clstst_df)

,origin_id,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,proximity_side_street_left_01,proximity_side_street_right_01
0,1,3,2.343517,0.898760,1,0
1,2,3,5.453073,2.941975,1,0
2,3,3,10.202465,5.621384,1,0
3,4,3,9.730853,5.831960,1,0
4,5,3,12.103868,6.461815,1,0
...,...,...,...,...,...,...
128,129,2,15.251560,9.303501,1,0
129,130,2,19.134858,12.312674,1,0
130,131,2,21.141576,13.032971,1,0
131,132,2,22.261617,14.728789,1,0
